**Official implementation of paper "Revisiting In-context Learning Inference Circuit in Large Language Models" (ICLR 2025)**:

### The Feature similarity on the Forerunner Token

This experiment is used to visualize the feature similarity on the forerunner token, to produce the Fig. 4 in the paper.

Author: Hakaze Cho, yfzhao@jaist.ac.jp, 2024/09

Organized, commented, and modified by: Hakaze Cho, 2025/01/27

**Part I: Import, Define, and Load Everything**

What you should do:
1. [Cell 1] Change to the path from your working directory to the directory containing the README.md file.
2. [Cell 2] Define your experiment parameters.
3. Run the Cell 1 and Cell 2.

In [ ]:
# Cell 1: Import libraries and change the working directory.

## Change the working directory
import os
try:
    # Change to the path from your working directory to the directory containing the README.md file.
    os.chdir("ICL_Inference_Dynamics_Released") 
except:
    print("Already in the correct directory or the directory does not exist.")

## Import libraries
from util import load_model_and_data, inference
import matplotlib.pyplot as plt
import StaICC
import numpy as np
from numpy.linalg import norm
import seaborn as sns

## Some definations for the plots.
plt.style.use('default')
plt.rc('font',family='Cambria Math')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Cambria Math'] + plt.rcParams['font.serif']

In [12]:
# Cell 2: Model and huggingfacetoken configurations

## The huggingface model name to be tested as the LM for ICL. 
## Recommended: "meta-llama/Meta-Llama-3-8B", "tiiuae/falcon-7b", "meta-llama/Meta-Llama-3-70B", "tiiuae/falcon-40b"
ICL_model_name = "tiiuae/falcon-7b" 

## Whether to use the quantized version of the model. 
## Recommended: Keep it default.
quantized = False if ICL_model_name in ["meta-llama/Meta-Llama-3-8B", "EleutherAI/pythia-6.9b", "tiiuae/falcon-7b"] else True

## The huggingface token to access the model. If you use the Llama model, you need to set this.
huggingface_token = "your token here"


# Experiment parameters

## The demonstration numbers. Recommended: 0, 1, 2, 4, 8, 12.
k = 4 

## The used dataset index from the StaICC library. Alternative: 0, 1, 2, 3, 4, 5. See the README.md for more information.
dataset_index = 0 

## The ks used in the axises for the heatmap. Recommended: [0, 1, 2, 4, 8, 12].
ks = [0, 1, 2, 4, 8, 12]

## Force the ICL_model to reload, even the ICL_model is already in the variables. 
## Recommended: False.
model_forced_reload = False

## Layer to be used in calculating the similarity. 
## Default: 16 for "meta-llama/Meta-Llama-3-8B" and "tiiuae/falcon-7b"; 24 for "tiiuae/falcon-40b" and "meta-llama/Meta-Llama-3-70B". 
layer_used = 16

In [ ]:
# Cell 3: Load the models.

vars_dict = vars() if "ICL_model" in vars() else locals()
if "ICL_model" not in vars_dict or model_forced_reload:
    ICL_model, ICL_tknz = load_model_and_data.load_ICL_model(ICL_model_name, huggingface_token = huggingface_token, quantized = quantized)
    loaded = True

**Part II: Run the Experiment**

What you should do:

1. Run the Cell 4.

In [ ]:
# Cell 4: Collect the ICL hidden states in the forerunner tokens for the different k values.

ICL_hidden_states_indexed_by_k = {}

for k in ks:
    print("k = ", k)
    bench = StaICC.Normal(k)
    prompts, queries = load_model_and_data.load_data_from_StaICC_experimentor(bench[dataset_index], "none")
    ICL_hidden_states = inference.ICL_inference_to_hidden_states(ICL_model, ICL_tknz, prompts)
    ICL_hidden_states_indexed_by_k[k] = ICL_hidden_states

In [ ]:
ICL_hidden_states_indexed_by_k

**Part III: Plot and Save the Result**

What you should do:

1. Run the Cell 5 - 7. You can define your own file name and dictionary to save the result in Cell 8.

In [ ]:
# Cell 5: Plot the blue heatmap with the same query.

heat_map_content_with_the_same_query = np.zeros((len(ks), len(ks)))

for i in range(len(ks)):
    for j in range(i + 1):
        temp = []
        for sample_index in range(len(ICL_hidden_states_indexed_by_k[ks[i]][layer_used])):
            A = ICL_hidden_states_indexed_by_k[ks[i]][layer_used][sample_index]
            B = ICL_hidden_states_indexed_by_k[ks[j]][layer_used][sample_index]
            temp.append(np.dot(A,B)/(norm(A)*norm(B)))
        heat_map_content_with_the_same_query[i][j] = np.mean(temp)

mask = []
for i in range(len(heat_map_content_with_the_same_query)):
    temp_mask = [0] * (i+1) + [1] * (len(heat_map_content_with_the_same_query)-i-1)
    mask.append(temp_mask)

plt.figure(figsize=(3, 3))
sns.heatmap(
    heat_map_content_with_the_same_query, 
    vmin = 0,
    vmax = 1,
    mask = np.array(mask), 
    annot=True, 
    cbar=False,
    linewidths=0.5,
    cmap = "PuBu",
    xticklabels = ks,
    yticklabels = ks
)
plt.xlabel("Query Location", fontsize = 12)
plt.ylabel("Query Location", fontsize = 12)
plt.title("Feature similarity on dataset " + str(dataset_index) + "\n model: " + ICL_model_name, fontsize = 12)

In [ ]:
# Cell 6: Plot the orange heatmap with the different query.

heat_map_content_with_the_same_query = np.zeros((len(ks), len(ks)))

for i in range(len(ks)):
    for j in range(i + 1):
        temp = []
        for sample_index in range(len(ICL_hidden_states_indexed_by_k[ks[i]][layer_used])):
            print('\r' + str(i) + str(j) + str(sample_index), flush=True, end="")
            for sample_index2 in range(len(ICL_hidden_states_indexed_by_k[ks[i]][layer_used])):
                if sample_index == sample_index2:
                    continue
                A = ICL_hidden_states_indexed_by_k[ks[i]][layer_used][sample_index]
                B = ICL_hidden_states_indexed_by_k[ks[j]][layer_used][sample_index2]
                temp.append(np.dot(A,B)/(norm(A)*norm(B)))
        heat_map_content_with_the_same_query[i][j] = np.mean(temp)

mask = []
for i in range(len(heat_map_content_with_the_same_query)):
    temp_mask = [0] * (i+1) + [1] * (len(heat_map_content_with_the_same_query)-i-1)
    mask.append(temp_mask)

plt.figure(figsize=(3, 3))
sns.heatmap(
    heat_map_content_with_the_same_query, 
    vmin = 0,
    vmax = 1,
    mask = np.array(mask), 
    annot=True, 
    cbar=False,
    linewidths=0.5,
    cmap = "Oranges",
    xticklabels = ks,
    yticklabels = ks
)
plt.xlabel("Query Location", fontsize = 12)
plt.ylabel("Query Location", fontsize = 12)
plt.title("Feature similarity on dataset " + str(dataset_index) + "\n model: " + ICL_model_name, fontsize = 12)

In [21]:
# Cell 7: Save the hidden states to a pickle file.
# Result file organization:
# ICL_hidden_states_indexed_by_k[k] is a list of hidden states, with index aligned with the sample number, for the k-th query location.

import pickle

data_file_name = "data/" + ICL_model_name.replace('/', '_') + ",ICLFeatures," + str(k) + ',' + str(dataset_index + 1) + ".pickle"
with open(data_file_name, 'wb') as f:
    pickle.dump(ICL_hidden_states_indexed_by_k, f)